#Sahibinden.com sitesi üzerinden Bursa'nın ilçelerindeki ev bilgilerini ilçeler bazında toplayacagız en son bu topladığımız bilgileri birleştireceğiz

In [ ]:
#gerekli kütüphanelerin kurulumu
import pyautogui
from selenium import webdriver
import time
import pandas as pd

#listeden string ifadeye çevirmek için kullanacağımız fonksiyon
def LisToString(s):
    str1 = " "
    return (str1.join(s))
#Chrome Webdriveri çağırıyoruz
browser = webdriver.Chrome()
"""
----------------------------------------------------------------------------------------------------------------------------------
sitedeki sayfa sayıları 50'nin katları şekilden değişiyor (2.sayfanın numarası 50, 3.sayfanın numarası 100...) bu yüzden sayfaları 
geçebilemk için iki değişken oluşturduk m sayfanın "pagingOffset=" kısmına gelecek şekilde ayarlandı. n her sayfadaki işlem bittikten
sonra bir sonraki sayfaya geçebilmek için bir arttırılarak devam ediyor.
----------------------------------------------------------------------------------------------------------------------------------
"""
n=1
m=0
while m<=100:
    #ilk sayfanın  "pagingOffset" kısmı 0 da başlıyor döngüyü bozmamsı için ona ayrı bir if yapısı uyguladım.
    if m==0:
        browser.get("https://www.sahibinden.com/satilik/bursa-iznik?pagingOffset="+str(m)+"&pagingSize=50")
        m+=1
        continue
    else:
        browser.get("https://www.sahibinden.com/satilik/bursa-iznik?pagingOffset="+str(m)+"&pagingSize=50")
        m=n*50
        print(n,"sayfa tamamlandi")
        n+=1
        
        time.sleep(2)

        
""""
------------------------------------------------------------------------------------------------------------------------------------
Her bir sayfanın içinde 50 adet ilan var bu ilanlar sıralı olarak tr[] şeklinde değişiyor. Sadece 4. tr[] reklam içerdiği için onu 
if yapısı ile devre dışı bıraktık.
------------------------------------------------------------------------------------------------------------------------------------
""""
        i = 1
        while i <= 51:
            if   i==4 :
                i+=1
                continue
            else:
                tıkla = browser.find_element_by_xpath("/html/body/div[4]/div[4]/form/div/div[3]/table/tbody/tr["+str(i)+"]")
                tıkla.click()
                
                #tıkladıktan sonra biraz bekletmek iyi oluyor çünkü tıkladıktan sonra sayfa hemen yüklenemezse bilgileri bulamayıp hata veriyor
                time.sleep(2)
                
                i+=1
                #evin konum bilgisi ayrı bir yerde verildiği için onu ayrı almamız gerekiyor
                konumlar=[]
                konum=browser.find_elements_by_css_selector("#classifiedDetail > div > div.classifiedDetailContent > div.classifiedInfo > h2 > a:nth-child(3)")
                for a in konum:
                    konumlar.append(a.text)
                    #print(a.text)
                    
                #evin fiyat bilgisi ayrı bir yerde verildiği için onu ayrı almamız gerekiyor
                fiyatlar=[]    
                fiyat=browser.find_elements_by_css_selector("#classifiedDetail > div > div.classifiedDetailContent > div.classifiedInfo > h3")
                for a in fiyat:
                    fiyatlar.append(a.text)
                    #print(a.text)
                    
                #evin detaylar bilgisi ayrı bir yerde verildiği için onu ayrı almamız gerekiyor
                detaylar=[]   
                detay=browser.find_elements_by_css_selector("#classifiedDetail > div > div.classifiedDetailContent > div.classifiedInfo > ul")
                for a in detay:
                    detaylar.append(a.text)
                    #print(a.text)
                #gelen detay bilgilerini daha kolay düzletmek için fonksiyonumuzu kullanıyoruz. 
                det_str = LisToString(detaylar)
                ayrı = det_str.split("\n")
                df = pd.DataFrame(ayrı)
                #bize sadece detay bilgileri lazım olduğu için başlıkları silmemiz lazım bunun için aşağıdaki fonksiyonu kullanıyoruz
                sinir=df.count()[0]-1
                j = 0
                while j <= sinir:
                    df.drop([j], axis=0 , inplace=True)
                    j = j + 2
                #başlıkları sildiğimiz için indexlerde kayma oldu bunu düzeltmek için de indexleri resetledik
                df_yeni = df.reset_index()    
                df_yeni.drop("index", axis = 1, inplace = True)
                df_liste = df_yeni.values.tolist()
                
                #fiaytlar TL olarak geldiğini bildiğimiz için sonundaki TL ifadesini siliyoruz
                yeni_fiyat=[]
                yeni_fiyat1=fiyatlar[0].replace("TL","").strip()
                yeni_fiyat.append(yeni_fiyat1)

                
                #oluşturduğumuz bilgileri listelere ekliyoruz
                df_liste.append(konumlar)
                df_liste.append(yeni_fiyat)
                
                #listenin Transpozunu alarak dataframe haline getiriyoruz
                df_içerikler = pd.DataFrame(df_liste).T
                #oluşturduğumuz dataframeyi csv dosyasına kaydediyoruz
                df_içerikler.to_csv(r"yildirim.csv",encoding="utf-8",index=False, mode="a")


                #ilan bilgilerini aldıktan sonra geri çıkma işlemini yapıyoruz
                browser.execute_script("window.history.go(-1)")
                #sayfanın tekrar yüklenmesini bekliyoruz aksi takdirde sayfa geç yüklenirse hata verebilir
                time.sleep(2)
        

### verileri ilçe ilçe indirmiştik bu yüzden gelen verileri birleştirmek gerekiyor

In [ ]:
#veriyi csv dosyasından okuyoruz ve uzunluğunu ölçüyoruz

In [ ]:
df=pd.read_csv("gemlik_1.csv")
uzunluk=len(df)-1

In [ ]:
#veriyi csv dosyasına her kaydedişte satırlara kendisi yeniden sütun oluşturduğu için bu satırdaki sütun bilgierini silmek gerekiyor

In [ ]:
i = 1
while i <= uzunluk:
    df.drop(i,inplace = True)
    i=i+2
df

In [ ]:
#satır silme işlemi yaptığımız için indexler bozuluyor bunu düzeltmek için de indexleri resetliyoruz

In [ ]:
df = df.reset_index()
df.drop("index", axis = 1, inplace = True)
df

In [ ]:
#ilçelerdeki bilgileri tek bir csv dosyasında birleştirmek için önce hepsini yüklüyoruz

In [ ]:
df_gemlik=pd.read_csv("gemlik_1.csv")
df_gursu=pd.read_csv("gursu_1.csv")
df_inegol=pd.read_csv("inegol_1.csv")
df_karacabey=pd.read_csv("karacabey_1.csv")
df_kestel=pd.read_csv("kestel_1.csv")
df_mudanya=pd.read_csv("mudanya_1.csv")
df_mustafakemalpasa=pd.read_csv("mustafakemalpasa_1.csv")
df_nilufer=pd.read_csv("nilufer_1.csv")
df_orhaneli=pd.read_csv("orhaneli_1.csv")
df_orhangazi=pd.read_csv("orhangazi_1.csv")
df_osmangazi=pd.read_csv("osmangazi_1.csv")
df_yenisehir=pd.read_csv("yenisehir_1.csv")
df_yildirim=pd.read_csv("yıldırım_1.csv")

In [ ]:
#concat metodu ile tüm ilçeleri birleştiriyoruz. indexler kaydığı için yine resetleme işlemi yapıyoruz

In [ ]:
df_enson=pd.concat([df_gemlik,df_gursu,df_inegol,df_karacabey,df_kestel,df_mudanya,df_mustafakemalpasa,
                   df_nilufer,df_orhaneli,df_orhangazi,df_osmangazi,df_yenisehir,df_yildirim]
                   axis=0)
df = df_enson.reset_index()
df.drop("index", axis = 1, inplace = True)
df

In [ ]:
#sütun adlarını düzenliyoruz

In [ ]:
df = df.rename(columns = {"0":"ilan_no",
                     "1":"ilan_tarihi",
                     "2":"emlak_tipi",
                     "3":"m2_brut",
                     "4":"m2_net",
                     "5":"oda_sayisi",
                     "6":"bina_yaşi",
                     "7":"bulundugu_kat",
                     "8":"kat_sayisi",
                     "9":"isitma",
                     "10":"banyo_sayisi",
                     "11":"balkon",
                     "12":"esyali",
                     "13":"kullanim_durumu",
                     "14":"site_icerisi",
                     "15":"site_adi",
                     "16":"aidat_tl",
                     "17":"krediye_uygun",
                     "18":"tapu_durumu",
                     "19":"kimden",
                     "20":"gezilebilir",
                     "21":"takas",
                     "22":"konum",
                     "23":"fiyat"})
df.head()

In [ ]:
#ilan no unique değer olduğu için onun üzerinden uniquelik durumunu gözlemleyip unique olmayan değerleri siliyoruz

In [ ]:
df["ilan_no"].count()

In [ ]:
df_dedupped = df.drop_duplicates()
df = df_dedupped.reset_index()
df.drop("index", axis = 1, inplace = True)
df["ilan_no"].count()

In [ ]:
df1=df.copy()

In [ ]:
df2=df1.dropna()

In [ ]:
df2.isnull().sum()

In [ ]:
df2 = df2.reset_index()
df2.drop("index", axis = 1, inplace = True)

In [ ]:
#son halini alan veri setini kaydediyoruz

In [ ]:
df2.to_csv(r"tamveri.csv",encoding="utf-8",index=False, mode="a")

In [ ]:
df2.to_excel("tamveri.xlsx")

# veriyi indirme ve birleştirme işlemleri tamamlandı